In [475]:
from scipy.optimize import minimize
from scipy.optimize import Bounds
from scipy.optimize import linprog
import pandas as pd
import numpy as np
import os
import json
from math import e

In [250]:
data_dir = '../data'

In [705]:
nutrition_profile_df = pd.DataFrame(index=['RDI', 'UL', 'target_fac', 'target_scale'], data={
    "Calories (kcal)": [2000, 2200, 1.0, 100.0],
    "Carbohydrates (g)": [279, 300, 1.0, 100.0],
    "Protein (g)": [104, 300, 1.0, 100.0],
    "Total Fat (g)": [57, 70, 1.0, 100.0],
    "Saturated Fat (g)": [0, None, 1.0, 0.0],
    "Monounsaturated Fat (g)": [0, None, 1.0, 0.0],
    "Polyunsaturated Fat (g)": [0, None, 1.0, 0.0],
    "Omega-3 Fatty Acids (g)": [8.5, None, 1.2, 1.0],
    "Omega-6 Fatty Acids (g)": [12, None, 1.2, 1.0],
    "Total Fiber (g)": [38, None, 1.2, 1.0],
    "Soluble Fiber (g)": [3, None, 1.2, 1.0],
    "Insoluble Fiber (g)": [30, None, 1.2, 1.0],
    "Cholesterol (mg)": [0, None, 1.0, 0.0],
    "Vitamin A (IU)": [3000, 10000, 1.2, 1.0],
    "Vitamin B6 (mg)": [1.3, 100, 1.2, 1.0],
    "Vitamin B12 (ug)": [2.4, None, 1.2, 1.0],
    "Vitamin C (mg)": [90, 2000, 1.2, 1.0],
    "Vitamin D (IU)": [600, 4000, 1.2, 1.0],
    "Vitamin E (IU)": [20, 1000, 1.2, 1.0],
    "Vitamin K (ug)": [120, None, 1.2, 1.0],
    "Thiamin (mg)": [1.2, None, 1.2, 1.0],
    "Riboflavin (mg)": [1.3, None, 1.2, 1.0],
    "Niacin (mg)": [16, 41, 1.2, 1.0],
    "Folate (ug)": [400, 1000, 1.2, 1.0],
    "Pantothenic Acid (mg)": [5, None, 1.2, 1.0],
    "Biotin (ug)": [30, None, 1.2, 1.0],
    "Choline (mg)": [550, 3500, 1.2, 1.0],
    "Calcium (g)": [1, 2.5, 1.2, 1.0],
    "Chloride (g)": [2.3, None, 1.2, 1.0],
    "Chromium (ug)": [35, None, 1.0, 1.0],
    "Copper (mg)": [0.9, 10, 1.0, 1.0],
    "Iodine (ug)": [150, 1100, 1.2, 1.0],
    "Iron (mg)": [8, 45, 1.2, 1.0],
    "Magnesium (mg)": [420, None, 1.2, 1.0], 
    "Manganese (mg)": [2.3, 11, 1.0, 1.0],
    "Molybdenum (ug)": [45, 2000, 1.2, 1.0],
    "Phosphorus (g)": [0.7, 4, 1.2, 1.0],
    "Potassium (g)": [4.7, None, 1.2, 1.0],
    "Selenium (ug)": [55, 400, 1.2, 1.0],
    "Sodium (g)": [1.5, 2.3, 1.2, 1.0],
    "Sulfur (g)": [0.8, 2, 1.2, 1.0],
    "Zinc (mg)": [11, 40, 1.2, 1.0],
    "Cystine (mg)": [816, 1000000, 2.0, 1.0],
    "Histidine (mg)": [780, None, 2.0, 1.0],
    "Isoleucine (mg)": [650, None, 2.0, 1.0],
    "Leucine (mg)": [910, None, 2.0, 1.0],
    "Lysine (mg)": [2176, 300000, 2.0, 1.0],
    "Methionine (mg)": [884, 80000, 2.0, 1.0],
    "Phenylalanine (mg)": [910, None, 2.0, 1.0],
    "Threonine (mg)": [455, 100000, 2.0, 1.0],
    "Tryptophan (mg)": [812, 400000, 2.0, 1.0],
    "Valine (mg)": [650, None, 2.0, 1.0],
    "Tyrosine (mg)": [875, None, 2.0, 0.0],
    "Arginine (mg)": [400, 2000, 2.0, 0.0],
    "Alanine (mg)": [200, None, 2.0, 0.0],
    "Aspartic acid (mg)": [500, 2000, 2.0, 0.0],
    "Glutamic acid (mg)": [500, 3000, 2.0, 0.0],
    "Glycine (mg)": [1000, 10000, 2.0, 0.0],
    "Proline (mg)": [500, None, 2.0, 0.0],
    "Serine (mg)": [500, None, 2.0, 0.0],
    "Hydroxyproline (mg)": [1, None, 2.0, 0.0]
})
#nutrition_profile_df = nutrition_profile_df.loc[4] = np.zeros(nutrition_profile_df.shape[1]).reshape(1,42), index=['target']))
#np.zeros(nutrition_profile_df.shape[1]).reshape(1,42)
#nutrition_profile_df.shape
#pd.DataFrame(np.zeros(nutrition_profile_df.shape[1]), index=['target'])
nutrition_profile_df.loc['target'] = nutrition_profile_df.loc['RDI'] * nutrition_profile_df.loc['target_fac']
#nutrition_profile_df.append(pd.Series(np.zeros(nutrition_profile_df.shape[1]).reshape(1,42)))
#nutrition_profile_df.append(pd.DataFrame(np.zeros(nutrition_profile_df.shape[1]).reshape(1,61), columns=nutrition_profile_df.columns, index=['target']))
#nutrition_profile_df.head(5)
#nutrition_profile_df.transpose().to_csv(f'{data_dir}/raw/nutrition_profile.csv')
nutrition_profile_df = pd.read_csv(f'{data_dir}/raw/nutrition_profile.csv', index_col=0, verbose=True).transpose()
nutrition_profile_df.head()

Tokenization took: 0.07 ms
Type conversion took: 0.35 ms
Parser memory cleanup took: 0.00 ms


,Calories (kcal),Carbohydrates (g),Protein (g),Total Fat (g),Saturated Fat (g),Monounsaturated Fat (g),Polyunsaturated Fat (g),Omega-3 Fatty Acids (g),Omega-6 Fatty Acids (g),Total Fiber (g),Soluble Fiber (g),Insoluble Fiber (g),Cholesterol (mg),Vitamin A (IU),Vitamin B6 (mg),Vitamin B12 (ug),Vitamin C (mg),Vitamin D (IU),Vitamin E (IU),Vitamin K (ug),Thiamin (mg),Riboflavin (mg),Niacin (mg),Folate (ug),Pantothenic Acid (mg),Biotin (ug),Choline (mg),Calcium (g),Chloride (g),Chromium (ug),Copper (mg),Iodine (ug),Iron (mg),Magnesium (mg),Manganese (mg),Molybdenum (ug),Phosphorus (g),Potassium (g),Selenium (ug),Sodium (g),Sulfur (g),Zinc (mg),Cystine (mg),Histidine (mg),Isoleucine (mg),Leucine (mg),Lysine (mg),Methionine (mg),Phenylalanine (mg),Threonine (mg),Tryptophan (mg),Valine (mg),Tyrosine (mg),Arginine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg)
RDI,2000.0,279.0,104.0,57.0,0.0,0.0,0.0,10.2,14.4,45.6,3.6,36.0,0.0,3600.0,1.6,2.9,108.0,1020.0,34.0,204.0,2.0,2.2,27.2,680.0,5.0,30.0,935.0,1.0,2.3,35.0,0.9,150.0,8.0,420.0,2.3,45.0,0.7,4.7,55.0,1.5,0.8,11.0,1632.0,1560.0,1300.0,1820.0,4352.0,1768.0,1820.0,910.0,1624.0,1300.0,875.0,400.0,200.0,500.0,500.0,1000.0,500.0,500.0,1.0
UL,2200.0,300.0,300.0,70.0,NaN,NaN,NaN,100.0,100.0,400.0,300.0,300.0,NaN,10000.0,100.0,240.0,2000.0,4000.0,1000.0,1200.0,50.0,50.0,41.0,1000.0,500.0,500.0,3500.0,2.5,500.0,350.0,10.0,1100.0,45.0,10000.0,11.0,2000.0,4.0,50.0,400.0,2.3,2.0,40.0,1000000.0,1000000.0,1000000.0,1000000.0,300000.0,80000.0,1000000.0,100000.0,400000.0,1000000.0,1000000.0,2000.0,1000000.0,2000.0,3000.0,10000.0,1000000.0,1000000.0,1000000.0
target_fac,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.2,1.2,1.2,1.2,1.2,1.0,1.2,1.2,1.2,1.2,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.0,1.0,1.7,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
target_scale,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
RDI Base,2000.0,279.0,104.0,57.0,0.0,0.0,0.0,8.5,12.0,38.0,3.0,30.0,0.0,3000.0,1.3,2.4,90.0,600.0,20.0,120.0,1.2,1.3,16.0,400.0,5.0,30.0,550.0,1.0,2.3,35.0,0.9,150.0,8.0,420.0,2.3,45.0,0.7,4.7,55.0,1.5,0.8,11.0,816.0,780.0,650.0,910.0,2176.0,884.0,910.0,455.0,812.0,650.0,875.0,400.0,200.0,500.0,500.0,1000.0,500.0,500.0,1.0


In [432]:
def sigmoid_valley(x, l_limit, u_limit, target_scale):
    penalty = 0
    if l_limit == 0:
        l_limit_den = 1.0
    else:
        l_limit_den = l_limit

    if x < l_limit:
        penalty = 0
        
    l_diff = min(x - l_limit, 100)
    
    left = 1 / (1 + e**((l_diff)/l_limit_den))

    if u_limit is None or np.isnan(u_limit):
        u_limit = l_limit_den * 10
        u_diff = min(x - l_limit_den * 10, 100)
        penalty = 0
    else:
        u_diff = min(x - u_limit, 100)

    if x > u_limit:
        penalty = 0
        
    right = 1 / (1 + e**(-1*(u_diff)/u_limit))
    return((left + right + penalty) * target_scale)

In [456]:
def victory(x, l_limit, u_limit, target_scale):
    oob_slope = 10
    ib_slope = 1.0
    val = 0.0
    
    if u_limit is None or np.isnan(u_limit):
        u_limit = l_limit * 10
        ib_slope = 1.0
        
    l_diff = x - l_limit
    u_diff = x - u_limit
    
    if l_diff < 0.0:
        val = -l_diff * oob_slope
        return(val)
    
    if u_diff > 0.0:
        val = u_diff * oob_slope
        val = val + l_diff * ib_slope
        return(val)
    
    val = val + l_diff * ib_slope
    return(val)
        

In [458]:
victory(20, 10, 100, 1.0)

10.0

In [252]:
nutrition_profile_df.loc['target'] = nutrition_profile_df.loc['RDI'] * nutrition_profile_df.loc['target_fac']
nutrition_profile_df.head(5)

,Calories (kcal),Carbohydrates (g),Protein (g),Total Fat (g),Saturated Fat (g),Monounsaturated Fat (g),Polyunsaturated Fat (g),Omega-3 Fatty Acids (g),Omega-6 Fatty Acids (g),Total Fiber (g),...,Valine (mg),Tyrosine (mg),Arginine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg)
RDI,2000.0,279.0,104.0,57.0,0.0,0.0,0.0,8.5,12.0,38.0,...,650.0,875.0,400.0,200.0,500.0,500.0,1000.0,500.0,500.0,1.0
UL,2200.0,300.0,300.0,70.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2000.0,NaN,2000.0,3000.0,10000.0,NaN,NaN,NaN
target_fac,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.2,1.2,1.2,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
target_scale,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
target,2000.0,279.0,104.0,57.0,0.0,0.0,0.0,10.2,14.4,45.6,...,1300.0,1750.0,800.0,400.0,1000.0,1000.0,2000.0,1000.0,1000.0,2.0


In [253]:

field_map = {
    'Name': 8,
    'Container Size (g)': 12,
    'Cost ($)': 16,
    'Source': 18,
    'Link': 20,
    'Calories (kcal)': 32,
    'Carbohydrates (g)': 35,
    "Protein (g)": 38,
    "Total Fat (g)": 41,
    "Saturated Fat (g)": 44,
    "Monounsaturated Fat (g)": 47,
    "Polyunsaturated Fat (g)": 50,
    "Omega-3 Fatty Acids (g)": 53,
    "Omega-6 Fatty Acids (g)": 56,
    "Total Fiber (g)": 59,
    "Soluble Fiber (g)": 62,
    "Insoluble Fiber (g)": 65,
    "Cholesterol (mg)": 68,
    "Vitamin A (IU)": 116,
    "Vitamin B6 (mg)": 119,
    "Vitamin B12 (ug)": 122,
    "Vitamin C (mg)": 125,
    "Vitamin D (IU)": 128,
    "Vitamin E (IU)": 131,
    "Vitamin K (ug)": 134,
    "Thiamin (mg)": 137,
    "Riboflavin (mg)": 140,
    "Niacin (mg)": 143,
    "Folate (ug)": 146,
    "Pantothenic Acid (mg)": 149,
    "Biotin (ug)": 152,
    "Choline (mg)": 155,
    "Calcium (g)": 71,
    "Chloride (g)": 74,
    "Chromium (ug)": 77,
    "Copper (mg)": 80,
    "Iodine (ug)": 83,
    "Iron (mg)": 86,
    "Magnesium (mg)": 89, 
    "Manganese (mg)": 92,
    "Molybdenum (ug)": 95,
    "Phosphorus (g)": 98,
    "Potassium (g)": 101,
    "Selenium (ug)": 104,
    "Sodium (g)": 107,
    "Sulfur (g)": 110,
    "Zinc (mg)": 113
}

ingredients = {}
for file in os.listdir(f'{data_dir}/raw'):

    if not file.endswith('.txt'):
        continue
    #print(f"name of file: {file}")        
    lines = open(f'{data_dir}/raw/{file}', 'r').readlines()

    name = lines[8].strip()
    ingred = {}
    
    for k, v in field_map.items():
        if k.endswith(')'):
            ingred[k] = float(lines[v].strip())
        else:
            ingred[k] = lines[v].strip()
    ingredients[name] = ingred

open(f'{data_dir}/interim/ingredients.json', 'w').write(json.dumps(ingredients, indent=4))
#print(str(ingredients))


31865

In [277]:
#ingredient_labels=list(ingredients[list(ingredients.keys())[0]].keys())[5:]
#ingredient_names=list(ingredients.keys())
#data = {}
#for label in ingredient_labels:
#    data[label]=[]
    
#for name in ingredient_names:
#    for label in ingredient_labels:
#        data[label].append(ingredients[name][label])
        
#ingredients_df = pd.DataFrame(index=ingredient_names, data=data)
#ingredients_df.to_json(open(f'{data_dir}/interim/ingredients_df.json','w'))
#ingredients_df.transpose().to_csv(f'{data_dir}/interim/ingredients_df.csv')
ingredients_df = pd.read_csv(f'{data_dir}/raw/Ugrainium Recipe Calculator - Ingredients.csv', index_col=0, verbose=True).transpose()
ingredients_df.head()
#ingredients_df['Calories (kcal)'].dtypes
for col in ingredients_df.columns:
    ingredients_df[col] = ingredients_df[col].astype(float)
ingredients_df.dtypes
ingredients_df.loc['Canola oil']
pd.set_option('max_columns', 70)
display(ingredients_df)

Tokenization took: 0.07 ms
Type conversion took: 0.77 ms
Parser memory cleanup took: 0.00 ms


,Calories (kcal),Carbohydrates (g),Protein (g),Total Fat (g),Saturated Fat (g),Monounsaturated Fat (g),Polyunsaturated Fat (g),Omega-3 Fatty Acids (g),Omega-6 Fatty Acids (g),Total Fiber (g),Soluble Fiber (g),Insoluble Fiber (g),Cholesterol (mg),Vitamin A (IU),Vitamin B6 (mg),Vitamin B12 (ug),Vitamin C (mg),Vitamin D (IU),Vitamin E (IU),Vitamin K (ug),Thiamin (mg),Riboflavin (mg),Niacin (mg),Folate (ug),Pantothenic Acid (mg),Biotin (ug),Choline (mg),Calcium (g),Chloride (g),Chromium (ug),Copper (mg),Iodine (ug),Iron (mg),Magnesium (mg),Manganese (mg),Molybdenum (ug),Phosphorus (g),Potassium (g),Selenium (ug),Sodium (g),Sulfur (g),Zinc (mg),Cystine (mg),Histidine (mg),Isoleucine (mg),Leucine (mg),Lysine (mg),Methionine (mg),Phenylalanine (mg),Threonine (mg),Tryptophan (mg),Valine (mg),Tyrosine (mg),Arginine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg)
Guar gum,332.0,77.30,4.60,0.50,0.060,0.220,0.070,0.001,0.067,77.3,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.000,0.0,0.0,0.294,0.000,0.0,0.000,0.0,0.00,0.0,0.000,0.0,0.000,0.000,0.0,0.125,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Vitamin K,0.0,0.00,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000,500.0,0.000,0.000,0.000,0.0,0.000,0.0,0.0,0.000,0.000,0.0,0.000,0.0,0.00,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Oat flour,400.0,65.00,17.50,7.50,1.600,2.900,3.300,0.145,3.185,10.0,4.0,6.0,0.0,0.0,0.125,0.0,0.0,0.0,1.000,3.2,0.700,0.125,1.500,32.0,0.200,0.0,29.9,0.065,0.000,0.0,0.437,0.0,4.00,144.0,4.000,0.0,0.452,0.371,34.0,0.019,0.000,3.20,408.0,405.0,694.0,1284.0,701.0,312.0,895.0,575.0,234.0,937.0,564.0,1179.0,872.0,1436.0,3718.0,846.0,923.0,744.0,0.0
Choline Bitartrate,0.0,0.00,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.000,0.0,550.0,0.000,0.000,0.0,0.000,0.0,0.00,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Wheat bran, crude",216.0,64.51,15.55,4.25,0.630,0.637,2.212,0.167,2.039,42.8,0.0,42.8,0.0,9.0,1.303,0.0,0.0,0.0,1.490,1.9,0.523,0.577,13.578,79.0,2.181,0.0,74.4,0.073,0.000,0.0,0.998,0.0,10.57,611.0,5.000,0.0,1.013,1.182,77.6,0.002,0.605,7.27,371.0,430.0,486.0,928.0,600.0,234.0,595.0,500.0,282.0,726.0,436.0,1087.0,765.0,1130.0,2874.0,898.0,882.0,684.0,0.0
Lecithin granules,700.0,10.00,0.00,50.00,15.000,0.000,35.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.000,0.0,13000.0,0.200,0.000,0.0,0.000,0.0,3.60,0.0,0.000,0.0,3.000,1.600,0.0,0.000,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alpha-galactosidase enzyme (Beano),0.0,0.00,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.000,0.0,0.0,0.000,0.000,0.0,0.000,0.0,0.00,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Canola oil,884.0,0.00,0.00,100.00,7.365,63.276,28.142,9.137,19.005,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,17.460,71.3,0.000,0.000,0.000,0.0,0.000,0.0,0.2,0.000,0.000,0.0,0.000,0.0,0.00,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pea Protein Powder,375.0,1.00,82.10,7.14,3.570,0.000,0.000,0.000,0.000,0.5,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.000,0.0,95.0,0.143,0.000,0.0,0.000,0.0,5.79,0.0,0.000,0.0,0.000,0.000,0.0,1.200,0.000,0.00,870.0,2510.0,4760.0,8410.0,7360.0,980.0,5520.0,4050.0,1030.0,5030.0,2810.0,6540.0,3210.0,9110.0,13530.0,3080.0,3380.0,4170.0,0.0
Multivitamin,0.0,0.00,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,3500.0,2.000,6.0,90.0,400.0,30.000,25.0,1.500,1.700,20.000,500.0,10.00

In [262]:
def objective(recipe):
    # recipe: contains the proportion of each ingredient measured in fractions of 100g
    
    # Find the total amount of all nutrients for the given recipe
    total = np.zeros(len(ingredients_df.values[0]))
    for i, amount in [ (i, np.transpose(recipe)[i]) for i in range(len(recipe))]:
        #print(str(total))
        ingredient = ingredients_df.values[i]
        scaled = ingredient * amount
        total[i] = scaled
        total[i] = sigmoid_valley(scaled, nutrition_profile_df)
    
    # Find the difference between the total and the target
    target = nutrition_profile_df.loc['target'].values
    target_scale = nutrition_profile_df.loc['target_scale'].values
    delta_vec = np.absolute(target - total)
    delta_vec = delta_vec * target_scale
    # TODO: Elliminate masked nutrients
    # TODO: may want to apply weights per nutrient
    #print('delta: ' + str(delta))
    
    # Calculate the sum of the differences
    error = np.sum(delta_vec)
    
    return(error)
        

In [392]:
r = list(range(1,22, 1))
r = (np.random.randn(20) + 2) * 0.01
r.shape
r = [0.009, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.009, 1.0, 1.00000001]
ingredients_df.index
re = pd.DataFrame(r, index=ingredients_df.index)
re.shape
ingredients_df.shape
scaled_ing_df = ingredients_df.mul(r, axis=0)
scaled_nut = scaled_ing_df.sum()
for j in list(zip(scaled_nut, nutrition_profile_df.loc['RDI'], nutrition_profile_df.loc['UL'])):
    print('args: ' + str(j) + ', val: ' + str(sigmoid_valley(*j)))
    #print(j)
#np.sum([sigmoid_valley(*j) for j in list(zip(scaled_nut, nutrition_profile_df.loc['RDI'], nutrition_profile_df.loc['UL']))])

args: (4535.16230534, 2000.0, 2200.0), val: 0.9988642837346264
args: (332.85640031, 279.0, 300.0), val: 0.9792438686130793
args: (323.92040023, 104.0, 300.0), val: 0.7964933876863243
args: (216.524500346, 57.0, 70.0), val: 0.9541734833873129
args: (32.766540037, 0.0, nan), val: 0.9069249887689909
args: (75.620980075, 0.0, nan), val: 0.9985890752359629
args: (100.18363028699999, 0.0, nan), val: 0.9998788503725314
args: (32.42200922813, 8.5, nan), val: 0.40666140380031385
args: (32.76860305911, 12.0, nan), val: 0.47636505130450546
args: (100.536400264, 38.0, nan), val: 0.48569583559512297
args: (20.440700131999996, 3.0, nan), val: 0.4239842785111233
args: (69.60000013199999, 30.0, nan), val: 0.5277301979787674
args: (0.0, 0.0, nan), val: 0.7689414213699951
args: (3723.0, 3000.0, 10000.0), val: 0.839699676754067
args: (4.824000005, 1.3, 100.0), val: 0.3408718805760796
args: (6.0, 2.4, nan), val: 0.5032468246309634
args: (1090.600000006, 90.0, 2000.0), val: 0.6359076829197116
args: (800.0,

In [459]:
def objective(r):
    # Scale the ingredient nutrient amounts for the given quantity of each ingredient given by the optimizer
    scaled_ing_df = ingredients_df.mul(r, axis=0)
    
    # Sum the total amounts of each nutrient for all of the ingredients scaled by the optimizer
    scaled_nut = scaled_ing_df.sum()
    
    # Calculate the fitness error 
    obj_val = np.sum([victory(*j) for j in list(zip(scaled_nut, nutrition_profile_df.loc['RDI'], nutrition_profile_df.loc['UL'], nutrition_profile_df.loc['target_scale']))])
    
    return(obj_val)

In [600]:
bounds_df = pd.DataFrame(index=ingredients_df.index, data={'lower': 0, 'upper': None})
bounds_df.loc['Guar gum'] = [0.9 * .01, 0.9 * .01]
bounds_df.loc['Xanthan Gum'] = [0.9 * .01, 0.9 * .01]
#bounds_df.loc['Vitamin K'] = [1.0, 1.0]
bounds_df.loc['Alpha-galactosidase enzyme (Beano)'] = [1.0, 1.0]
bounds_df.loc['Multivitamin'] = [1.0, 1.0]
bounds_df.loc['Corn flour, nixtamalized'] = [0, 1.0]
bounds_df.loc['Whey protein'] = [0,0.15]
#bounds_df.loc['Calcium and vitamin D'] = [1.0, 1.0]
#ingredients_df.index
bounds = list(zip(bounds_df.lower.values, bounds_df.upper.values))
bounds

[(0.009000000000000001, 0.009000000000000001),
 (0.0, None),
 (0.0, None),
 (0.0, None),
 (0.0, None),
 (0.0, None),
 (1.0, 1.0),
 (0.0, None),
 (0.0, None),
 (1.0, 1.0),
 (0.0, 1.0),
 (0.0, None),
 (0.0, None),
 (0.0, None),
 (0.0, 0.15),
 (0.0, None),
 (0.0, None),
 (0.009000000000000001, 0.009000000000000001),
 (0.0, None),
 (0.0, None)]

In [461]:
# First pass using the general solver
rec_len = ingredients_df.shape[0]
rec = (np.random.randn(rec_len) + 2) * 0.01
solution = minimize(objective, rec, bounds=bounds)
solution

      fun: 51210.81515700641
 hess_inv: <20x20 LbfgsInvHessProduct with dtype=float64>
      jac: array([-1252.14646687,   500.00016927,  9458.57318584,   550.00054999,
       -4804.29443996, 14688.59991292,     0.        ,  3187.71562888,
       11883.62584799, -3329.21845256,  1900.89267562, 62239.99880604,
        1000.00033854, 10779.05981219,  5662.93601878,  -182.00007617,
       -4005.9996536 , -2096.08333535,  -476.39987315,  5551.3184634 ])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 4011
      nit: 43
   status: 0
  success: True
        x: array([0.009     , 0.15534167, 0.        , 0.00380652, 0.51067931,
       0.03894221, 1.        , 2.09535306, 0.1132526 , 0.95027497,
       0.        , 0.        , 0.03798483, 0.        , 0.        ,
       0.06392463, 0.54929205, 0.009     , 0.06393672, 0.        ])

In [759]:
nutrition_profile_df

,Calories (kcal),Carbohydrates (g),Protein (g),Total Fat (g),Saturated Fat (g),Monounsaturated Fat (g),Polyunsaturated Fat (g),Omega-3 Fatty Acids (g),Omega-6 Fatty Acids (g),Total Fiber (g),Soluble Fiber (g),Insoluble Fiber (g),Cholesterol (mg),Vitamin A (IU),Vitamin B6 (mg),Vitamin B12 (ug),Vitamin C (mg),Vitamin D (IU),Vitamin E (IU),Vitamin K (ug),Thiamin (mg),Riboflavin (mg),Niacin (mg),Folate (ug),Pantothenic Acid (mg),Biotin (ug),Choline (mg),Calcium (g),Chloride (g),Chromium (ug),Copper (mg),Iodine (ug),Iron (mg),Magnesium (mg),Manganese (mg),Molybdenum (ug),Phosphorus (g),Potassium (g),Selenium (ug),Sodium (g),Sulfur (g),Zinc (mg),Cystine (mg),Histidine (mg),Isoleucine (mg),Leucine (mg),Lysine (mg),Methionine (mg),Phenylalanine (mg),Threonine (mg),Tryptophan (mg),Valine (mg),Tyrosine (mg),Arginine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg)
RDI,2000.0,279.0,104.0,57.0,0.0,0.0,0.0,10.2,14.4,45.6,3.6,36.0,0.0,3600.0,1.6,2.9,108.0,1020.0,34.0,204.0,2.0,2.2,27.2,680.0,5.0,30.0,935.0,1.7,2.3,35.0,0.9,150.0,13.6,420.0,2.3,45.0,1.2,8.0,55.0,1.5,1.4,18.7,1632.0,1560.0,1300.0,1820.0,4352.0,1768.0,1820.0,910.0,1624.0,1300.0,1750.0,400.0,400.0,500.0,500.0,2000.0,1000.0,1000.0,1.0
UL,2200.0,300.0,300.0,70.0,NaN,NaN,NaN,100.0,100.0,400.0,300.0,300.0,NaN,10000.0,100.0,240.0,2000.0,4000.0,1000.0,1200.0,50.0,50.0,41.0,1000.0,500.0,500.0,3500.0,2.5,500.0,350.0,10.0,1100.0,45.0,10000.0,11.0,2000.0,4.0,50.0,400.0,2.3,2.0,40.0,1000000.0,1000000.0,1000000.0,1000000.0,300000.0,80000.0,1000000.0,100000.0,400000.0,1000000.0,1000000.0,2000.0,1000000.0,2000.0,3000.0,10000.0,1000000.0,1000000.0,1000000.0
target_fac,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.2,1.2,1.2,1.2,1.2,1.0,1.2,1.2,1.2,1.2,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.0,1.0,1.7,1.7,1.0,1.0,1.0,1.0,1.7,1.0,1.0,1.0,1.7,1.7,1.0,1.0,1.7,1.7,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0
target_scale,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
RDI Base,2000.0,279.0,104.0,57.0,0.0,0.0,0.0,8.5,12.0,38.0,3.0,30.0,0.0,3000.0,1.3,2.4,90.0,600.0,20.0,120.0,1.2,1.3,16.0,400.0,5.0,30.0,550.0,1.0,2.3,35.0,0.9,150.0,8.0,420.0,2.3,45.0,0.7,4.7,55.0,1.5,0.8,11.0,816.0,780.0,650.0,910.0,2176.0,884.0,910.0,455.0,812.0,650.0,875.0,400.0,200.0,500.0,500.0,1000.0,500.0,500.0,1.0
UL Base,2200.0,300.0,300.0,70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10000.0,100.0,NaN,2000.0,4000.0,1000.0,NaN,NaN,NaN,41.0,1000.0,NaN,NaN,3500.0,2.5,NaN,NaN,10.0,1100.0,45.0,NaN,11.0,2000.0,4.0,NaN,400.0,2.3,2.0,40.0,1000000.0,NaN,NaN,NaN,300000.0,80000.0,NaN,100000.0,400000.0,NaN,NaN,2000.0,NaN,2000.0,3000.0,10000.0,NaN,NaN,NaN
target_fac_try,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.2,1.2,1.2,1.2,1.2,1.0,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.0,1.0,1.7,1.7,1.7,1.0,1.7,1.7,1.7,1.7,1.0,1.7,1.7,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4


In [772]:
# Using the linprog optimizer
#!mv ~/Downloads/Ugrainium\ Recipe\ Calculator\ -\ Nutrition\ Profile.csv nutrition_profile.csv
!(cd ..; make sync_data_from_google)

nutrition_profile_df = pd.read_csv(f'{data_dir}/raw/nutrition_profile.csv', index_col=0, verbose=True).transpose()
# Define the cost function. This can be actual cost of ingredients or a measure of the non-objectionability of ingredients to minimize
c = np.ones(ingredients_df.shape[0])

# Define the inequality constraint matrix. This is the ingredients_df data in two parts, the first
# for constraining nutrients to be greater than the RDI, the second to constrain to be less than the UL

A = -ingredients_df.transpose()[nutrition_profile_df.loc['target_scale'].ne(0)]
A = A.append(-A).values


# The upper bound inequality constraint vector. This is the nutrient UL (upper limit)

b = -nutrition_profile_df.loc['RDI'][nutrition_profile_df.loc['target_scale'].ne(0)]
b = b.append(nutrition_profile_df.loc['UL'][nutrition_profile_df.loc['target_scale'].ne(0)]).values


solution = linprog(c, A_ub=A, b_ub=b, bounds=bounds)
solution


curl 'https://docs.google.com/spreadsheets/d/1UOx5ZkNMjcwybx9TgFS_EaQnjbokqDx6K9NW0mRDGlg/export?format=csv&id=1UOx5ZkNMjcwybx9TgFS_EaQnjbokqDx6K9NW0mRDGlg&gid=624419712' > data/raw/nutrition_profile.csv
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2975    0  2975    0     0   8548      0 --:--:-- --:--:-- --:--:--  8548
Tokenization took: 0.03 ms
Type conversion took: 0.92 ms
Parser memory cleanup took: 0.01 ms


     con: array([], dtype=float64)
     fun: 9.260953962095506
 message: 'Optimization terminated successfully.'
     nit: 27
   slack: array([1.58936946e+02, 3.76303433e-10, 6.41270221e+00, 1.29593405e+01,
       1.38855265e+01, 9.33084721e-11, 4.87912115e-01, 2.06163725e+01,
       8.24018542e-09, 4.16116563e-08, 1.54561225e+00, 1.90000000e+00,
       4.65649919e-09, 3.28077476e-09, 1.12313508e+00, 2.32279262e-09,
       2.79987412e+00, 1.50831947e-01, 1.22952245e+01, 4.03974908e+01,
       3.48082255e+00, 0.00000000e+00, 8.45532213e-08, 6.17550455e-12,
       4.52919709e+00, 0.00000000e+00, 3.14435465e+00, 2.32087748e+00,
       2.66074861e+01, 5.27762553e+02, 8.70000000e+00, 0.00000000e+00,
       1.20511561e+00, 2.43872478e-10, 1.41052420e+02, 1.83406401e-10,
       5.38768875e-01, 6.15745994e+00, 3.47784610e+02, 9.86104177e+02,
       3.64691562e+03, 6.67899525e+03, 1.57997169e+03, 8.07134801e+01,
       3.52468544e+03, 3.65325684e+03, 6.01949068e-09, 4.34077561e+03,
       1.666

In [699]:

#nutrition_profile_df.loc['target_scale'].ne(0).shape

Ap = -ingredients_df.transpose()[nutrition_profile_df.loc['target_scale'].ne(0)]
Ap = Ap.append(-Ap)

bp = -nutrition_profile_df.loc['RDI'][nutrition_profile_df.loc['target_scale'].ne(0)]
bp = bp.append(nutrition_profile_df.loc['UL'][nutrition_profile_df.loc['target_scale'].ne(0)])

#NP = nutrition_profile_df.copy()
#NP.transpose()[pd.isna(nutrition_profile_df.loc['UL'])]
#= NP.transpose()[pd.isna(nutrition_profile_df.loc['UL'])]['RDI']

In [773]:
# Scale the ingredient nutrient amounts for the given quantity of each ingredient given by the optimizer
solution_df = ingredients_df.mul(solution.x, axis=0)
solution_df.insert(0, 'Quantity (g)', solution.x * 100)

# Add a row showing the sum of the scaled amount of each nutrient
total = solution_df.sum()
total.name = 'Total'
solution_df = solution_df.append(total)
solution_df

,Quantity (g),Calories (kcal),Carbohydrates (g),Protein (g),Total Fat (g),Saturated Fat (g),Monounsaturated Fat (g),Polyunsaturated Fat (g),Omega-3 Fatty Acids (g),Omega-6 Fatty Acids (g),Total Fiber (g),Soluble Fiber (g),Insoluble Fiber (g),Cholesterol (mg),Vitamin A (IU),Vitamin B6 (mg),Vitamin B12 (ug),Vitamin C (mg),Vitamin D (IU),Vitamin E (IU),Vitamin K (ug),Thiamin (mg),Riboflavin (mg),Niacin (mg),Folate (ug),Pantothenic Acid (mg),Biotin (ug),Choline (mg),Calcium (g),Chloride (g),Chromium (ug),Copper (mg),Iodine (ug),Iron (mg),Magnesium (mg),Manganese (mg),Molybdenum (ug),Phosphorus (g),Potassium (g),Selenium (ug),Sodium (g),Sulfur (g),Zinc (mg),Cystine (mg),Histidine (mg),Isoleucine (mg),Leucine (mg),Lysine (mg),Methionine (mg),Phenylalanine (mg),Threonine (mg),Tryptophan (mg),Valine (mg),Tyrosine (mg),Arginine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg)
Guar gum,9.000000e-01,2.988000e+00,6.957000e-01,4.140000e-02,4.500000e-03,5.400000e-04,0.001980,0.000630,0.000009,0.000603,6.957000e-01,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000e+00,2.646000e-03,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.125000e-03,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
Vitamin K,3.119080e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,155.954017,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
Oat flour,6.844925e+01,2.737970e+02,4.449201e+01,1.197862e+01,5.133694e+00,1.095188e+00,1.985028,2.258825,0.099251,2.180109,6.844925e+00,2.737970,4.106955,0.0,0.000000,0.085562,0.0,0.000000,0.0,0.684493,2.190376,0.479145,0.085562,1.026739,21.903761,0.136899,0.0,2.046633e+01,4.449201e-02,0.000000,0.0,0.299123,0.000000,2.737970e+00,98.566924,2.737970,0.0,0.309391,0.253947,23.272746,1.300536e-02,0.000000,2.190376,2.792730e+02,2.772195e+02,4.750378e+02,8.788884e+02,4.798293e+02,2.135617e+02,6.126208e+02,3.935832e+02,1.601713e+02,6.413695e+02,3.860538e+02,8.070167e+02,5.968775e+02,9.829313e+02,2.544943e+03,5.790807e+02,6.317866e+02,5.092624e+02,0.000000
Choline Bitartrate,4.802621e-10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,2.641441e-09,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
"Wheat bran, crude",7.874326e-01,1.700854e+00,5.079728e-01,1.224458e-01,3.346589e-02,4.960825e-03,0.005016,0.017418,0.001315,0.016056,3.370211e-01,0.000000,0.337021,0.0,0.070869,0.010260,0.0,0.000000,0.0,0.011733,0.014961,0.004118,0.004543,0.106918,0.622072,0.017174,0.0,5.858498e-01,5.748258e-04,0.000000,0.0,0.007859,0.000000,8.323162e-02,4.811213,0.039372,0.0,0.007977,0.009307,0.611048,1.574865e-05,0.004764,0.057246,2.921

In [626]:
nutrition_profile_df

,Calories (kcal),Carbohydrates (g),Protein (g),Total Fat (g),Saturated Fat (g),Monounsaturated Fat (g),Polyunsaturated Fat (g),Omega-3 Fatty Acids (g),Omega-6 Fatty Acids (g),Total Fiber (g),Soluble Fiber (g),Insoluble Fiber (g),Cholesterol (mg),Vitamin A (IU),Vitamin B6 (mg),Vitamin B12 (ug),Vitamin C (mg),Vitamin D (IU),Vitamin E (IU),Vitamin K (ug),Thiamin (mg),Riboflavin (mg),Niacin (mg),Folate (ug),Pantothenic Acid (mg),Biotin (ug),Choline (mg),Calcium (g),Chloride (g),Chromium (ug),Copper (mg),Iodine (ug),Iron (mg),Magnesium (mg),Manganese (mg),Molybdenum (ug),Phosphorus (g),Potassium (g),Selenium (ug),Sodium (g),Sulfur (g),Zinc (mg),Cystine (mg),Histidine (mg),Isoleucine (mg),Leucine (mg),Lysine (mg),Methionine (mg),Phenylalanine (mg),Threonine (mg),Tryptophan (mg),Valine (mg),Tyrosine (mg),Arginine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg)
RDI,2000.0,279.0,104.0,57.0,0.0,0.0,0.0,14.5,20.4,64.6,5.1,51.0,0.0,5100.0,2.2,4.1,153.0,1020.0,34.0,204.0,2.0,2.2,27.2,680.0,8.5,51.0,935.0,1.7,3.9,35.0,0.9,150.0,8.0,420.0,2.3,45.0,0.7,4.7,55.0,1.5,0.8,11.0,816.0,780.0,650.0,910.0,2176.0,884.0,910.0,455.0,812.0,650.0,875.0,400.0,200.0,500.0,500.0,1000.0,500.0,500.0,1.0
UL,2200.0,300.0,300.0,70.0,NaN,NaN,NaN,100.0,100.0,400.0,300.0,300.0,NaN,10000.0,100.0,240.0,2000.0,4000.0,1000.0,1200.0,50.0,50.0,41.0,1000.0,500.0,500.0,3500.0,2.5,500.0,350.0,10.0,1100.0,45.0,10000.0,11.0,2000.0,4.0,50.0,400.0,2.3,2.0,40.0,1000000.0,1000000.0,1000000.0,1000000.0,300000.0,80000.0,1000000.0,100000.0,400000.0,1000000.0,1000000.0,2000.0,1000000.0,2000.0,3000.0,10000.0,1000000.0,1000000.0,1000000.0
target_fac,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.7,1.7,1.7,1.7,1.7,1.0,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
target_scale,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
RDI Base,2000.0,279.0,104.0,57.0,0.0,0.0,0.0,8.5,12.0,38.0,3.0,30.0,0.0,3000.0,1.3,2.4,90.0,600.0,20.0,120.0,1.2,1.3,16.0,400.0,5.0,30.0,550.0,1.0,2.3,35.0,0.9,150.0,8.0,420.0,2.3,45.0,0.7,4.7,55.0,1.5,0.8,11.0,816.0,780.0,650.0,910.0,2176.0,884.0,910.0,455.0,812.0,650.0,875.0,400.0,200.0,500.0,500.0,1000.0,500.0,500.0,1.0
UL Base,2200.0,300.0,300.0,70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10000.0,100.0,NaN,2000.0,4000.0,1000.0,NaN,NaN,NaN,41.0,1000.0,NaN,NaN,3500.0,2.5,NaN,NaN,10.0,1100.0,45.0,NaN,11.0,2000.0,4.0,NaN,400.0,2.3,2.0,40.0,1000000.0,NaN,NaN,NaN,300000.0,80000.0,NaN,100000.0,400000.0,NaN,NaN,2000.0,NaN,2000.0,3000.0,10000.0,NaN,NaN,NaN
target_fac_try,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.7,1.7,1.7,1.7,1.7,1.0,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.0,1.0,1.7,1.7,1.7,1.0,1.7,1.7,1.7,1.7,1.0,1.7,1.7,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4


In [463]:
ingredients_df

,Calories (kcal),Carbohydrates (g),Protein (g),Total Fat (g),Saturated Fat (g),Monounsaturated Fat (g),Polyunsaturated Fat (g),Omega-3 Fatty Acids (g),Omega-6 Fatty Acids (g),Total Fiber (g),Soluble Fiber (g),Insoluble Fiber (g),Cholesterol (mg),Vitamin A (IU),Vitamin B6 (mg),Vitamin B12 (ug),Vitamin C (mg),Vitamin D (IU),Vitamin E (IU),Vitamin K (ug),Thiamin (mg),Riboflavin (mg),Niacin (mg),Folate (ug),Pantothenic Acid (mg),Biotin (ug),Choline (mg),Calcium (g),Chloride (g),Chromium (ug),Copper (mg),Iodine (ug),Iron (mg),Magnesium (mg),Manganese (mg),Molybdenum (ug),Phosphorus (g),Potassium (g),Selenium (ug),Sodium (g),Sulfur (g),Zinc (mg),Cystine (mg),Histidine (mg),Isoleucine (mg),Leucine (mg),Lysine (mg),Methionine (mg),Phenylalanine (mg),Threonine (mg),Tryptophan (mg),Valine (mg),Tyrosine (mg),Arginine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg)
Guar gum,332.0,77.30,4.60,0.50,0.060,0.220,0.070,0.001,0.067,77.3,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.000,0.0,0.0,0.294,0.000,0.0,0.000,0.0,0.00,0.0,0.000,0.0,0.000,0.000,0.0,0.125,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Vitamin K,0.0,0.00,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000,500.0,0.000,0.000,0.000,0.0,0.000,0.0,0.0,0.000,0.000,0.0,0.000,0.0,0.00,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Oat flour,400.0,65.00,17.50,7.50,1.600,2.900,3.300,0.145,3.185,10.0,4.0,6.0,0.0,0.0,0.125,0.0,0.0,0.0,1.000,3.2,0.700,0.125,1.500,32.0,0.200,0.0,29.9,0.065,0.000,0.0,0.437,0.0,4.00,144.0,4.000,0.0,0.452,0.371,34.0,0.019,0.000,3.20,408.0,405.0,694.0,1284.0,701.0,312.0,895.0,575.0,234.0,937.0,564.0,1179.0,872.0,1436.0,3718.0,846.0,923.0,744.0,0.0
Choline Bitartrate,0.0,0.00,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.000,0.0,550.0,0.000,0.000,0.0,0.000,0.0,0.00,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Wheat bran, crude",216.0,64.51,15.55,4.25,0.630,0.637,2.212,0.167,2.039,42.8,0.0,42.8,0.0,9.0,1.303,0.0,0.0,0.0,1.490,1.9,0.523,0.577,13.578,79.0,2.181,0.0,74.4,0.073,0.000,0.0,0.998,0.0,10.57,611.0,5.000,0.0,1.013,1.182,77.6,0.002,0.605,7.27,371.0,430.0,486.0,928.0,600.0,234.0,595.0,500.0,282.0,726.0,436.0,1087.0,765.0,1130.0,2874.0,898.0,882.0,684.0,0.0
Lecithin granules,700.0,10.00,0.00,50.00,15.000,0.000,35.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.000,0.0,13000.0,0.200,0.000,0.0,0.000,0.0,3.60,0.0,0.000,0.0,3.000,1.600,0.0,0.000,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alpha-galactosidase enzyme (Beano),0.0,0.00,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.000,0.0,0.0,0.000,0.000,0.0,0.000,0.0,0.00,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Canola oil,884.0,0.00,0.00,100.00,7.365,63.276,28.142,9.137,19.005,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,17.460,71.3,0.000,0.000,0.000,0.0,0.000,0.0,0.2,0.000,0.000,0.0,0.000,0.0,0.00,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pea Protein Powder,375.0,1.00,82.10,7.14,3.570,0.000,0.000,0.000,0.000,0.5,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.000,0.0,95.0,0.143,0.000,0.0,0.000,0.0,5.79,0.0,0.000,0.0,0.000,0.000,0.0,1.200,0.000,0.00,870.0,2510.0,4760.0,8410.0,7360.0,980.0,5520.0,4050.0,1030.0,5030.0,2810.0,6540.0,3210.0,9110.0,13530.0,3080.0,3380.0,4170.0,0.0
Multivitamin,0.0,0.00,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,3500.0,2.000,6.0,90.0,400.0,30.000,25.0,1.500,1.700,20.000,500.0,10.00